In [2]:
from SpinBiSaturation.BiSaturation import BiSaturation
import numpy as np
from bvpdae_solver import BVPDAE, BVPSol
from timeit import default_timer as dtime
from utils_functions import plot_sequence
import matplotlib.pyplot as plt
import pickle

In [14]:
def main_primal_dual():
    cumul_time = 0.
    eps = .1
    alpha = 0.5
    tol = 1e-7
    ocp = BiSaturation()
    ocp.set_eps(eps)
    options = dict(control_odes_error=True, display=1, res_tol=1e-3)
    bvp_solver = BVPDAE(**options)
    time, xp, z = ocp.initialize()
    initial_solution = dict(time=time, xp=xp, z=z)
    convergence = False
    iter = 0
    times, x0s, x1s, x2s, x3s, p0s, p1s, p2s, p3s, us = [list() for _ in range(10)]
    bvp_sol = BVPSol(time=time, xp=xp, z=z)
    while not convergence:
        print("eps = ", eps)
        print("alpha = ", alpha)
        ts0 = dtime()
        bvp_sol, infos = bvp_solver.solve(bvp_sol, ocp)
        print("infos NLSE = ", infos.NLSE_infos.bc_residual)
        print("iter = ", iter)
        ts1 = dtime()
        cumul_time += ts1 - ts0
        if not infos.success:
            raise Exception("Primal-dual BiSaturation problem failed before convergence")
        time, xp, z = bvp_sol.time, bvp_sol.xp, bvp_sol.z
        convergence = eps <= tol
        eps *= alpha
        ocp.set_eps(eps)
        times.append(xp[4, 0] * time)
        x0s.append(xp[0, :])
        x1s.append(xp[1, :])
        x2s.append(xp[2, :])
        x3s.append(xp[3, :])
        p0s.append(xp[5, :])
        p1s.append(xp[6, :])
        p2s.append(xp[7, :])
        p3s.append(xp[8, :])
        us.append(z[0])
        iter += 1
        print(" ")
    optimal_solution = dict(time=time, xp=xp, z=z, eps=eps / alpha, alpha=alpha, exec_time=cumul_time, iter=iter)
    dict_save = dict(initial_solution=initial_solution, optimal_solution=optimal_solution)
    with open("SpinBiSaturation/results_BiSaturation_primal_dual.pickle", "wb") as fh:
        pickle.dump(dict_save, fh)
    plot_sequence(times, x0s, "$x_0(t)$",
                  "Sequence of optimal penalized $\\bar{x}_0$")
    plot_sequence(times, x1s, "$x_1(t)$",
                  "Sequence of optimal penalized $\\bar{x}_1$")
    plot_sequence(times, x2s, "$x_2(t)$",
                    "Sequence of optimal penalized $\\bar{x}_2$")
    plot_sequence(times, x3s, "$x_3(t)$",
                    "Sequence of optimal penalized $\\bar{x}_3$")
    plot_sequence(times, p0s, "$p_0(t)$",
                  "Sequence of optimal penalized $\\bar{p}_0$")
    plot_sequence(times, p1s, "$p_1(t)$",
                  "Sequence of optimal penalized $\\bar{p}_1$")
    plot_sequence(times, p2s, "$p_2(t)$",
                    "Sequence of optimal penalized $\\bar{p}_2$")
    plot_sequence(times, p3s, "$p_3(t)$",
                    "Sequence of optimal penalized $\\bar{p}_3$")
    plot_sequence(times, us, "$u(t)$",
                  "Sequence of optimal penalized $\\bar{u}_0$")
 

    t = np.linspace(0., 2. * np.pi, 1000)
    x1t = ocp.r * ocp.a1 * np.cos(t) + ocp.xfinal[0] / 2.
    x2t = ocp.r * ocp.a2 * np.sin(t) + ocp.xfinal[1] / 2.5
    plt.figure()
    plt.plot(xp[0], xp[1])
    plt.plot(x1t, x2t)

    plt.figure()

    print("Time final = ", xp[4, 0])
    print("Execution time = ", cumul_time)
    print("Number of iterations = ", len(times))
    print("Number of final time steps = ", len(time))
    plt.show()


if __name__ == "__main__":
    main_primal_dual()

eps =  0.1
alpha =  0.5
     # Residual error = 1.5873525112487923 with N = 501
     # Residual error = 7.962064672998207 with N = 635
     # Residual error = 250.48166379579965 with N = 1517
     # Residual error = 587.3089346827488 with N = 3816
     # Residual error = 2517.086341151253 with N = 9035
     # Residual error = 2530.1202551662727 with N = 20881
     # Residual error = 3158.3242768943396 with N = 49029
Solving complete
infos NLSE =  1.7202308334469312
iter =  0


Exception: Primal-dual BiSaturation problem failed before convergence